# Convert a HDF5 compressed dataset to sparse matrices

This notebook presents different ways to convert a stack of images stored as a HDF5 dataset to a stack of sparse matrices.

Notebook license: [CC-0](https://creativecommons.org/public-domain/cc0/)

In [1]:
# Disable multithreading
import os

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["BLOSC_NTHREADS"] = "1"

In [2]:
import b2h5py
import blosc2
import bslz4_to_sparse
import h5py
from hdf5plugin import Bitshuffle, Blosc2
import numpy as np

## Prepare the datasets

Download data (credits ID11@ESRF) and create datasets with different compressions.

In [3]:
if not os.path.exists("sparse_image_stack.h5"):
    !wget http://www.silx.org/pub/leaps-innov/sparse_image_stack.h5

In [4]:
if not os.path.exists("sparse_images.h5"):
    with h5py.File("sparse_image_stack.h5", "r") as h5f:
        data = h5f["entry_0000/measurement/data"][:1000]

    chunk_shape = (1,) + data.shape[1:]

    with h5py.File("sparse_images.h5", "w") as h5f:
        h5f.create_dataset(
            "bslz4",
            data=data,
            chunks=chunk_shape,
            compression=Bitshuffle(),
        )
        h5f.create_dataset(
            "bszstd",
            data=data,
            chunks=chunk_shape,
            compression=Bitshuffle(cname="zstd"),
        )
        h5f.create_dataset(
            "blosc2_bslz4",
            data=data,
            chunks=chunk_shape,
            compression=Blosc2(cname="lz4", filters=Blosc2.BITSHUFFLE),
        )
        h5f.create_dataset(
            "blosc2_bszstd",
            data=data,
            chunks=chunk_shape,
            compression=Blosc2(cname="zstd", filters=Blosc2.BITSHUFFLE),
        )
        
    del data

## Benchmark reading data

Compare different ways to read data from a HDf5 compressed dataset.

The test is done on the first frame of a 3D dataset

In [5]:
h5f = h5py.File("sparse_images.h5", "r")
bslz4_dataset = h5f["bslz4"]
bszstd_dataset = h5f["bszstd"]
blosc2_bslz4_dataset = h5f["blosc2_bslz4"]
blosc2_bszstd_dataset = h5f["blosc2_bszstd"]

In [6]:
h5_chunk_info = blosc2_bszstd_dataset.id.get_chunk_info(0)
b2_schunk = blosc2.schunk.open(
    blosc2_bszstd_dataset.file.filename,
    mode='r',
    offset=h5_chunk_info.byte_offset,
)
print(b2_schunk.info)

type    : NDArray
shape   : (1, 2162, 2068)
chunks  : (1, 2162, 2068)
blocks  : (1, 256, 256)
dtype   : |V4
cratio  : 1698.96
cparams : {'blocksize': 262144,
 'clevel': 5,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.BITSHUFFLE: 2>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 16,
 'splitmode': <SplitMode.FORWARD_COMPAT_SPLIT: 4>,
 'typesize': 4,
 'use_dict': 0}
dparams : {'nthreads': 16}



Compressed size of first frame

In [7]:
bslz4_nbytes = len(bslz4_dataset.id.read_direct_chunk((0, 0, 0))[1])
bszstd_nbytes = len(bszstd_dataset.id.read_direct_chunk((0, 0, 0))[1])
blosc2_bslz4_nbytes = len(blosc2_bslz4_dataset.id.read_direct_chunk((0, 0, 0))[1])
blosc2_bszstd_nbytes = len(blosc2_bszstd_dataset.id.read_direct_chunk((0, 0, 0))[1])
print(f"{bslz4_nbytes=}\n{bszstd_nbytes=}\n{blosc2_bslz4_nbytes=}\n{blosc2_bszstd_nbytes=}")

bslz4_nbytes=183882
bszstd_nbytes=111675
blosc2_bslz4_nbytes=102058
blosc2_bszstd_nbytes=12757


* **With hdf5plugin**: Decompression is performed by the HDF5 filters

In [8]:
%timeit bslz4_dataset[0]

9.81 ms ± 68.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit bszstd_dataset[0]

5.36 ms ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit blosc2_bslz4_dataset[0]

6.19 ms ± 32.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit blosc2_bszstd_dataset[0]

6.06 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* **With b2h5py**: Datasets are accessed with direct chunk read and decompressed with blosc2

In [12]:
blosc2_bslz4_b2dataset = b2h5py.B2Dataset(blosc2_bslz4_dataset)

%timeit blosc2_bslz4_b2dataset[0]

6.2 ms ± 277 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
blosc2_bszstd_b2dataset = b2h5py.B2Dataset(blosc2_bszstd_dataset)

%timeit blosc2_bszstd_b2dataset[0]

5.55 ms ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* **Direct chunk read**: For reference read compressed data from file

In [14]:
%timeit bslz4_dataset.id.read_direct_chunk((0, 0, 0))

4.18 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
%timeit bszstd_dataset.id.read_direct_chunk((0, 0, 0))

2.85 µs ± 1.86 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
%timeit blosc2_bslz4_dataset.id.read_direct_chunk((0, 0, 0))

2.65 µs ± 44.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [17]:
%timeit blosc2_bszstd_dataset.id.read_direct_chunk((0, 0, 0))

944 ns ± 2.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Convert data to sparse representation

Compares different ways to convert a 3D dataset to sparse matrices frame by frame.

The test is done on the first frame.

In [18]:
def array_to_sparse(array: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Convert an array to sparse representation"""
    array = np.ravel(array)
    indices = np.nonzero(array)[0]
    values = array[indices]
    return values, indices

* **Reference**: Read data through h5py and convert it with numpy

In [19]:
%timeit array_to_sparse(bslz4_dataset[0])

8.69 ms ± 8.73 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%timeit array_to_sparse(bszstd_dataset[0])

10 ms ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%timeit array_to_sparse(blosc2_bslz4_dataset[0])

10.8 ms ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit array_to_sparse(blosc2_bszstd_dataset[0])

10.5 ms ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


With b2h5py:

In [23]:
blosc2_bslz4_b2dataset = b2h5py.B2Dataset(blosc2_bslz4_dataset)

%timeit array_to_sparse(blosc2_bslz4_b2dataset[0])

10.6 ms ± 7.53 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
blosc2_bszstd_b2dataset = b2h5py.B2Dataset(blosc2_bszstd_dataset)

%timeit array_to_sparse(blosc2_bszstd_b2dataset[0])

10.3 ms ± 2.64 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* **With bslz4_to_sparse** for bitshuffle+LZ4 compressed dataset

In [25]:
%timeit bslz4_to_sparse.bslz4_to_sparse(bslz4_dataset, 0, 0)

3.71 ms ± 36.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* **With b2h5py** for blosc2 compressed datasets

In [26]:
def blosc2_to_sparse(
    b2dataset: b2h5py.B2Dataset,
    frame: int
) -> tuple[int, tuple[np.ndarray, np.ndarray]]:
    """Reads one frame of a 3D blsoc2 compressed dataset and converts to sparse representation
    """
    assert b2dataset.ndim == 3
    ndata = np.prod(b2dataset.shape[1:])
    values = np.empty((ndata,), dtype=b2dataset.dtype)
    indices = np.empty((ndata,), dtype=np.uint32)
    nvalues = 0

    # TODO align reading with blosc2 blocs which are 256x256
    nrows = 256
    row_length = b2dataset.shape[2]
    for row in range(0, b2dataset.shape[1], nrows):
        data = np.ravel(b2dataset[frame, row:row+nrows])
        partial_indices = np.nonzero(data)[0]
        previous_nvalues = nvalues
        nvalues += len(partial_indices)
        values[previous_nvalues:nvalues] = data[partial_indices]
        indices[previous_nvalues:nvalues] = partial_indices + (row * row_length)

    return nvalues, (values, indices)

In [27]:
blosc2_bslz4_b2dataset = b2h5py.B2Dataset(blosc2_bslz4_dataset)

%timeit blosc2_to_sparse(blosc2_bslz4_b2dataset, 0)

14.1 ms ± 23 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
blosc2_bszstd_b2dataset = b2h5py.B2Dataset(blosc2_bszstd_dataset)

%timeit blosc2_to_sparse(blosc2_bszstd_b2dataset, 0)

13.9 ms ± 43.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Tests

Check that all methods returns the same results

In [29]:
ref_values, ref_indices = array_to_sparse(bslz4_dataset[0])
ref_nvalues = len(ref_values)

In [30]:
blosc2_bslz4_b2dataset = b2h5py.B2Dataset(blosc2_bslz4_dataset)
bslz4_to_sparse_results = bslz4_to_sparse.bslz4_to_sparse(bslz4_dataset, 0, 0)

assert bslz4_to_sparse_results[0] == ref_nvalues
assert np.array_equal(bslz4_to_sparse_results[1][0][:ref_nvalues], ref_values)
assert np.array_equal(bslz4_to_sparse_results[1][1][:ref_nvalues], ref_indices)

In [31]:
blosc2_bszstd_b2dataset = b2h5py.B2Dataset(blosc2_bszstd_dataset)
blosc2_to_sparse_results = blosc2_to_sparse(blosc2_bszstd_b2dataset, 0)

assert blosc2_to_sparse_results[0] == ref_nvalues
assert np.array_equal(blosc2_to_sparse_results[1][0][:ref_nvalues], ref_values)
assert np.array_equal(blosc2_to_sparse_results[1][1][:ref_nvalues], ref_indices)

### Alternative implementation

In [32]:
def read_blosc2_h5_chunk(dataset: h5py.Dataset, chunk_index: int):
    h5_chunk_info = dataset.id.get_chunk_info(chunk_index)
    b2_schunk = blosc2.schunk.open(
        dataset.file.filename,
        mode='r',
        offset=h5_chunk_info.byte_offset,
    )
    return b2_schunk[:].view(dtype=dataset.dtype)

In [33]:
%timeit array_to_sparse(read_blosc2_h5_chunk(blosc2_bszstd_dataset, 0))

8.55 ms ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
blosc2_bszstd_b2dataset = b2h5py.B2Dataset(blosc2_bszstd_dataset)

%timeit array_to_sparse(blosc2_bszstd_b2dataset[0])

15.8 ms ± 47.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
# Check that result is correct
assert np.array_equal(
    blosc2_bszstd_dataset[0],
    read_blosc2_h5_chunk(blosc2_bszstd_dataset, 0).reshape(blosc2_bszstd_dataset[0].shape)
)